In [10]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [40]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input=28x28x1 output=28x28x -? OUtput? RF =3x3
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1) # RF=5x5
        self.bn2 = nn.BatchNorm2d(16) 
        self.pool1 = nn.MaxPool2d(2, 2) # RF = 10x10 
        self.dp1 =nn.Dropout(0.25)
        self.conv3 = nn.Conv2d(16, 24, 3, padding=1) # RF=12x12
        self.bn3 = nn.BatchNorm2d(24)
        self.conv4 = nn.Conv2d(24, 28, 3, padding=1) # RD=14x14
        self.bn4 = nn.BatchNorm2d(28)
        self.pool2 = nn.MaxPool2d(2, 2) # RF=28x28
        self.dp2 =nn.Dropout(0.25)
        self.conv5 = nn.Conv2d(28, 24, 3) # RF=30x30
        self.bn5 = nn.BatchNorm2d(24)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.conv6 = nn.Conv2d(24, 16, 3) # RF=32x32
        self.conv7 = nn.Conv2d(16, 10, 3)

    def forward(self, x):
        x = self.dp1(self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x))))))))
        x = self.dp2(self.pool2(F.relu(self.bn4(self.conv4(F.relu(self.bn3(self.conv3(x))))))))
        x = F.relu(self.conv6(F.relu(self.bn5( self.conv5(x)))))
        x = (self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [41]:
!pip install torchsummary
from torchsummary import summary


In [42]:
use_mps = torch.backends.mps.is_available()
print("Is mps available:{}".format(use_mps))
use_cuda = torch.device("mps" if use_mps else "cpu")
print(use_cuda)

Is mps available:True
mps


In [43]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [44]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nEpoch: {} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        epoch, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [45]:
use_mps = torch.backends.mps.is_available()
print("Is mps available:{}".format(use_mps))
device = torch.device("mps" if use_mps else "cpu")

Is mps available:True


In [46]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|                                                                                                                                                                                        | 0/469 [00:00<?, ?it/s]/var/folders/25/vqnx0dh53y7clwgprnzshhxr0000gn/T/ipykernel_84630/2704648177.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.15846781432628632 batch_id=468: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:06<00:00, 68.19it/s]



Epoch: 1 Test set: Average loss: 0.0465, Accuracy: 9845/10000 (98.4500%)



loss=0.06607086211442947 batch_id=468: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:05<00:00, 78.62it/s]



Epoch: 2 Test set: Average loss: 0.0338, Accuracy: 9889/10000 (98.8900%)



loss=0.028396014124155045 batch_id=468: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:05<00:00, 79.76it/s]


...


Epoch: 17 Test set: Average loss: 0.0149, Accuracy: 9949/10000 (99.4900%)



loss=0.038799017667770386 batch_id=468: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:06<00:00, 77.87it/s]



Epoch: 18 Test set: Average loss: 0.0152, Accuracy: 9954/10000 (99.5400%)



loss=0.006463752593845129 batch_id=468: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:06<00:00, 77.46it/s]



Epoch: 19 Test set: Average loss: 0.0152, Accuracy: 9950/10000 (99.5000%)



In [47]:
#use_mps = torch.backends.mps.is_available()
#print("Is mps available:{}".format(use_mps))
#device = torch.device("mps:0" if use_mps else "cpu")

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,168
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
           Dropout-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 24, 14, 14]           3,480
       BatchNorm2d-8           [-1, 24, 14, 14]              48
            Conv2d-9           [-1, 28, 14, 14]           6,076
      BatchNorm2d-10           [-1, 28, 14, 14]              56
        MaxPool2d-11             [-1, 28, 7, 7]               0
          Dropout-12             [-1, 28, 7, 7]               0
           Conv2d-13             [-1, 24, 5, 5]           6,072
      BatchNorm2d-14             [-1, 2

/var/folders/25/vqnx0dh53y7clwgprnzshhxr0000gn/T/ipykernel_84630/2704648177.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
